```shell

cd /public/workspace/licanchengup/link/disease/dataset/LungCancer_GSE127465
conda activate
jupyter nbconvert 1_run.ipynb --to python
conda activate env_came
conda env list

rm nohup.out
nohup python 1_run.py &

jobs

conda activate
```

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import scanpy as sc
from IPython.display import display

p_link = Path("/public/workspace/licanchengup/link")
p_disease = p_link.joinpath("disease")
p_test = p_link.joinpath("test")
p_disease_res = p_disease.joinpath("test_disease")
p_disease_res.mkdir(parents=True, exist_ok=True)
p_root = p_disease.joinpath("dataset", Path(".").absolute().name)
print("cd {}".format(p_root.absolute()))

cd /public/workspace/licanchengup/link/disease/dataset/LungCancer_GSE127465


In [2]:
import sys

if str(p_test) not in sys.path:
    sys.path.append(str(p_test))

from func import (
    get_test_result_df,
    group_agg,
    test_came,
    test_came_and_csMAHN,
    test_csMAHN,
)

Help on function get_counts_from_adatas in module csMAHN.utils.preprocess:

get_counts_from_adatas(adatas)
    Assum adatas has been normalized
    Get a pair of dataframes from a pair adatas
    
    :param adatas: a tuple of reference adata and query adata
    :return: a tuple of reference raw counts dataframe and query raw counts dataframe

Help on function get_counts_from_adatas in module func:

get_counts_from_adatas(adatas)
    补丁
    version = 0.0.3


[func]----------------------------------
group_agg
rm_rf
get_type_counts_info		aligned
aligned_type			aligned
test_came_and_csMAHN

[result tools]--------------------------
get_test_result_df
show_umap
----------------------------------------
    
[out] test_finish_info_240103.csv


,dir,name,tissue,sp_ref,sp_que,model,resdir_tag,umap_dataset,umap_umap,p_cell_type_table,p_cell_type_table_unalign,cell_count_ref,cell_count_que
0,/public/workspace/licanchengup/link/disease/te...,LungCancer_h-corss-m;came;GSE127465_h_t-map-GS...,LungCancer,human,mouse,came,GSE127465_h_t-map-GSE127465_m_t;DendriticCells...,/public/workspace/licanchengup/link/disease/te...,/public/workspace/licanchengup/link/disease/te...,/public/workspace/licanchengup/link/disease/te...,/public/workspace/licanchengup/link/disease/te...,913,570
1,/public/workspace/licanchengup/link/disease/te...,LungCancer_m-corss-h;csMAHN;GSE127465_m_t-map-...,LungCancer,mouse,human,csMAHN,GSE127465_m_t-map-GSE127465_h_t;DendriticCells...,/public/workspace/licanchengup/link/disease/te...,/public/workspace/licanchengup/link/disease/te...,/public/workspace/licanchengup/link/disease/te...,/public/workspace/licanchengup/link/disease/te...,570,913


(341, 13)

dir
name
tissue
sp_ref
sp_que
model
resdir_tag
umap_dataset
umap_umap
p_cell_type_table
p_cell_type_table_unalign
cell_count_ref
cell_count_que


In [3]:
row_h = {
    "name": "GSE127465_h_t",
    "h5ad": p_root.joinpath("human_t.h5ad"),
    "species": "human",
    "disease": "LungCancer",
    "tissue": "lung",
}

row_m = {
    "name": "GSE127465_m_t",
    "h5ad":  p_root.joinpath("mouse_t.h5ad"),
    "species": "mouse",
    "disease": "LungCancer",
    "tissue": "lung",
}

In [4]:
def limite_func(adata1, adata2, key, value):
    adata1 = adata1.copy()
    adata2 = adata2.copy()
    adata1 = adata1[adata1.obs[key].str.match(value)]
    adata2 = adata2[adata2.obs[key].str.match(value)]
    return (adata1, adata2)

limite_func_map = {
    "all": lambda adata1, adata2: (adata1, adata2),
    "Neutrophils": lambda adata1, adata2: limite_func(
        adata1, adata2, key="CellType", value="^Neutrophils$"
    ),
    "DendriticCells_noMonoDC": lambda adata1, adata2: limite_func(
        adata1, adata2, key="SubCellType", value="^p?DC"
    ),
    "DendriticCells": lambda adata1, adata2: limite_func(
        adata1, adata2, key="SubCellType", value="p?(Mono)?DC\d?"
    ),
    "Monocyte_withMonoDC": lambda adata1, adata2: limite_func(
        adata1, adata2, key="SubCellType", value="Mono"
    ),
    "Monocyte": lambda adata1, adata2: limite_func(
        adata1, adata2, key="SubCellType", value="Mono\d"
    )
}

display(limite_func_map)

# 测试limite_func_map
adata_h, adata_m = sc.read_h5ad(row_h["h5ad"]), sc.read_h5ad(row_m["h5ad"])
for limite_func_map_key in limite_func_map.keys():
    k = "CellType" if len(limite_func_map_key) == 0 else "SubCellType"
    print(limite_func_map_key.ljust(70, "-"))
    adata_1, adata_2 = limite_func_map[limite_func_map_key](adata_h, adata_m)
    display(
        pd.concat(
            {
                "h": adata_1.obs["SubCellType"].value_counts(sort=False),
                "m": adata_2.obs["SubCellType"].value_counts(sort=False),
            },
            axis=1,
        )
    )

{'': <function __main__.<lambda>(adata1, adata2)>,
 'Neutrophils': <function __main__.<lambda>(adata1, adata2)>,
 'DendriticCells_noMonoDC': <function __main__.<lambda>(adata1, adata2)>,
 'DendriticCells': <function __main__.<lambda>(adata1, adata2)>,
 'Monocyte_withMonoDC': <function __main__.<lambda>(adata1, adata2)>,
 'Monocyte': <function __main__.<lambda>(adata1, adata2)>}

----------------------------------------------------------------------


,h,m
SubCellType,,
B,5218.0,NaN
DC1,171.0,119.0
DC2,330.0,106.0
DC3,101.0,293.0
Mac1,949.0,149.0
Mac2,919.0,100.0
Mac3,2554.0,23.0
Mac4,418.0,131.0
Mac5,165.0,NaN


Neutrophils-----------------------------------------------------------


,h,m
SubCellType,,
N1,526.0,968
N2,86.0,24
N3,425.0,130
N4,789.0,1782
N5,902.0,580
N6,NaN,109


...

,h,m
SubCellType,,
DC1,171,119
DC2,330,106
DC3,101,293
MonoDC,1097,76
pDC,311,52


Monocyte_withMonoDC---------------------------------------------------


,h,m
SubCellType,,
Mono1,698,407
Mono2,252,154
Mono3,226,210
MonoDC,1097,76


Monocyte--------------------------------------------------------------


,h,m
SubCellType,,
Mono1,698,407
Mono2,252,154
Mono3,226,210


In [5]:
# human --> mouse
row_ref, row_que = row_h, row_m
path_varmap = p_test.joinpath("came_sample_data/gene_matches_human2mouse.csv")

print(row_ref["name"], "\t", row_que["name"])
print(row_ref["disease"], "\t", row_que["disease"])
print(row_ref["species"], "\t\t", row_que["species"])
print("[varmap] {}".format(path_varmap.name))



GSE127465_h_t 	 GSE127465_m_t
LungCancer 	 LungCancer
human 		 mouse
[varmap] gene_matches_human2mouse.csv


In [6]:
for limite_func_map_key in limite_func_map.keys():
    # limite_func_map_key = "Monocyte"
    k = "CellType" if len(limite_func_map_key) == 0 else "SubCellType"
    _, adatas = test_came_and_csMAHN(
        row_ref["h5ad"],
        row_que["h5ad"],
        key_class1=k,
        key_class2=k,
        sp1=row_ref["species"][0],
        sp2=row_que["species"][0],
        tissue_name=row_ref["disease"],  # 这个tissue不是分离的，当前缀用吧
        resdir_tag="{}-map-{};{}".format(row_ref["name"], row_que["name"],limite_func_map_key),
        path_varmap=path_varmap,
        resdir=p_disease_res,
        aligned=False,
        limite_func=limite_func_map[limite_func_map_key]
    )

[has finish] LungCancer_h-corss-m;came;GSE127465_h_t-map-GSE127465_m_t;
[has finish] LungCancer_h-corss-m;csMAHN;GSE127465_h_t-map-GSE127465_m_t;
[has finish] LungCancer_h-corss-m;came;GSE127465_h_t-map-GSE127465_m_t;Neutrophils
[has finish] LungCancer_h-corss-m;csMAHN;GSE127465_h_t-map-GSE127465_m_t;Neutrophils
[has finish] LungCancer_h-corss-m;came;GSE127465_h_t-map-GSE127465_m_t;DendriticCells_noMonoDC
[has finish] LungCancer_h-corss-m;csMAHN;GSE127465_h_t-map-GSE127465_m_t;DendriticCells_noMonoDC
[has finish] LungCancer_h-corss-m;came;GSE127465_h_t-map-GSE127465_m_t;DendriticCells
[has finish] LungCancer_h-corss-m;csMAHN;GSE127465_h_t-map-GSE127465_m_t;DendriticCells
[has finish] LungCancer_h-corss-m;came;GSE127465_h_t-map-GSE127465_m_t;Monocyte_withMonoDC
[has finish] LungCancer_h-corss-m;csMAHN;GSE127465_h_t-map-GSE127465_m_t;Monocyte_withMonoDC
[has finish] LungCancer_h-corss-m;came;GSE127465_h_t-map-GSE127465_m_t;Monocyte
[has finish] LungCancer_h-corss-m;csMAHN;GSE127465_h_t-m

In [7]:
# mouse --> human
row_ref, row_que =  row_m,row_h
path_varmap = p_test.joinpath("came_sample_data/gene_matches_mouse2human.csv")

print(row_ref["name"], "\t", row_que["name"])
print(row_ref["disease"], "\t", row_que["disease"])
print(row_ref["species"], "\t\t", row_que["species"])
print("[varmap] {}".format(path_varmap.name))

for limite_func_map_key in limite_func_map.keys():
    k = "CellType" if len(limite_func_map_key) == 0 else "SubCellType"
    _, adatas = test_came_and_csMAHN(
        row_ref["h5ad"],
        row_que["h5ad"],
        key_class1=k,
        key_class2=k,
        sp1=row_ref["species"][0],
        sp2=row_que["species"][0],
        tissue_name=row_ref["disease"],  # 这个tissue不是分离的，当前缀用吧
        resdir_tag="{}-map-{};{}".format(row_ref["name"], row_que["name"],limite_func_map_key),
        path_varmap=path_varmap,
        resdir=p_disease_res,
        aligned=False,
        limite_func=limite_func_map[limite_func_map_key]
    )

GSE127465_m_t 	 GSE127465_h_t
LungCancer 	 LungCancer
mouse 		 human
[varmap] gene_matches_mouse2human.csv
[has finish] LungCancer_m-corss-h;came;GSE127465_m_t-map-GSE127465_h_t;
[has finish] LungCancer_m-corss-h;csMAHN;GSE127465_m_t-map-GSE127465_h_t;
[has finish] LungCancer_m-corss-h;came;GSE127465_m_t-map-GSE127465_h_t;Neutrophils
[has finish] LungCancer_m-corss-h;csMAHN;GSE127465_m_t-map-GSE127465_h_t;Neutrophils
[has finish] LungCancer_m-corss-h;came;GSE127465_m_t-map-GSE127465_h_t;DendriticCells_noMonoDC
[has finish] LungCancer_m-corss-h;csMAHN;GSE127465_m_t-map-GSE127465_h_t;DendriticCells_noMonoDC
[has finish] LungCancer_m-corss-h;came;GSE127465_m_t-map-GSE127465_h_t;DendriticCells
[has finish] LungCancer_m-corss-h;csMAHN;GSE127465_m_t-map-GSE127465_h_t;DendriticCells
[has finish] LungCancer_m-corss-h;came;GSE127465_m_t-map-GSE127465_h_t;Monocyte_withMonoDC
[has finish] LungCancer_m-corss-h;csMAHN;GSE127465_m_t-map-GSE127465_h_t;Monocyte_withMonoDC
[has finish] LungCancer_m-cor

In [8]:
print("\n[finish]\n".center(100, "-"))

---------------------------------------------
[finish]
---------------------------------------------



## 减少epochs

In [9]:
help(test_csMAHN)

Help on function test_csMAHN in module func:

test_csMAHN(path_adata1, path_adata2, key_class1, key_class2, sp1, sp2, tissue_name, path_varmap, aligned=False, resdir_tag='.', resdir=PosixPath('/public/workspace/licanchengup/download/test/test_result'), limite_func=<function <lambda> at 0x7f5844cdd790>, **kvargs)
    kvargs:
        n_epochs:
            default,[100, 200, 300]
            stages,即res_0,res_1，res_2 的 epochs
            累加制，res_0,res_1，res_2,实际epochs分别为100,300,600
            故最终epochs为stages之和
            stages = kvargs.setdefault("n_epochs",[100, 200, 300])



In [10]:
# human --> mouse
row_ref, row_que = row_h, row_m
path_varmap = p_test.joinpath("came_sample_data/gene_matches_human2mouse.csv")

print(row_ref["name"], "\t", row_que["name"])
print(row_ref["disease"], "\t", row_que["disease"])
print(row_ref["species"], "\t\t", row_que["species"])
print("[varmap] {}".format(path_varmap.name))

limite_func_map_key = "all"# all cell type
k = "CellType" if len(limite_func_map_key) == 0 else "SubCellType"


test_csMAHN(
    row_ref["h5ad"],
    row_que["h5ad"],
    key_class1=k,
    key_class2=k,
    sp1=row_ref["species"][0],
    sp2=row_que["species"][0],
    tissue_name=row_ref["disease"],  # 这个tissue不是分离的，当前缀用吧
    resdir_tag="csMAHN;{}-map-{};;{}".format(row_ref["name"], row_que["name"],"epochs=100"),
    path_varmap=path_varmap,
    resdir=p_disease_res,
    aligned=False,
    limite_func=limite_func_map[limite_func_map_key],
    **{"stages" : [100,100,100]}
)

GSE127465_h_t 	 GSE127465_m_t
LungCancer 	 LungCancer
human 		 mouse
[varmap] gene_matches_human2mouse.csv
[has finish] LungCancer_h-corss-m;csMAHN;GSE127465_h_t-map-GSE127465_m_t;;epochs=100


In [11]:
# mouse --> human
row_ref, row_que = row_m, row_h
path_varmap = p_test.joinpath("came_sample_data/gene_matches_mouse2human.csv")

print(row_ref["name"], "\t", row_que["name"])
print(row_ref["disease"], "\t", row_que["disease"])
print(row_ref["species"], "\t\t", row_que["species"])
print("[varmap] {}".format(path_varmap.name))

limite_func_map_key = "all"# all cell type
k = "CellType" if len(limite_func_map_key) == 0 else "SubCellType"


test_csMAHN(
    row_ref["h5ad"],
    row_que["h5ad"],
    key_class1=k,
    key_class2=k,
    sp1=row_ref["species"][0],
    sp2=row_que["species"][0],
    tissue_name=row_ref["disease"],  # 这个tissue不是分离的，当前缀用吧
    resdir_tag="csMAHN;{}-map-{};;{}".format(row_ref["name"], row_que["name"],"epochs=100"),
    path_varmap=path_varmap,
    resdir=p_disease_res,
    aligned=False,
    limite_func=limite_func_map[limite_func_map_key],
    **{"stages" : [100,100,100]}
)

GSE127465_m_t 	 GSE127465_h_t
LungCancer 	 LungCancer
mouse 		 human
[varmap] gene_matches_mouse2human.csv
[has finish] LungCancer_m-corss-h;csMAHN;GSE127465_m_t-map-GSE127465_h_t;;epochs=100


In [12]:
print("\n[finish][sub epochs]\n".center(100, "-"))

---------------------------------------
[finish][sub epochs]
---------------------------------------
